In [1]:
from __future__ import annotations

import json
import os
from dataclasses import asdict
from pathlib import Path
from uuid import UUID

# Настройки запуска (правь тут)
CONFIG_PATH = Path("configs/dev/market_data.yaml")
# если хочешь другой env-файл — поменяй
ENV_PATHS = [Path("/etc/roehub/roehub.env")]

# Параметры catch-up
BATCH_SIZE = 10_000
INGEST_ID = UUID("00000000-0000-0000-0000-000000000001")  # поменяй при желании
MAX_DAYS_PER_INSERT = None  # None => возьмём из config.backfill.max_days_per_insert

# Режим запуска:
RUN_ALL_FROM_REF_INSTRUMENTS = False

# Для одиночного инструмента:
MARKET_ID = 1
SYMBOL = "BTCUSDT"

# Куда сохранить json-отчёт (None => не сохранять)
REPORT_JSON_PATH = Path("scripts/data/market_data/_reports/rest_catchup_report.json")


In [2]:
from dotenv import dotenv_values


def load_env_files(paths: list[Path]) -> None:
    loaded = 0
    for p in paths:
        if p.exists():
            vals = dotenv_values(p)
            # dotenv_values может вернуть None для строк без значения
            for k, v in vals.items():
                if v is None:
                    continue
                os.environ.setdefault(k, str(v))
            loaded += 1
    print(f"Loaded env files: {loaded}/{len(paths)}")
    # покажем только ключи (без значений)
    keys = sorted(set().union(*[set(dotenv_values(p).keys()) for p in paths if p.exists()]))
    print("Env keys:", keys)

load_env_files(ENV_PATHS)


Loaded env files: 0/1
Env keys: []


In [3]:
from apps.cli.wiring.db.clickhouse import ClickHouseSettingsLoader, _clickhouse_client
from trading.contexts.market_data.adapters.outbound.config.runtime_config import (
    load_market_data_runtime_config,
)
from trading.contexts.market_data.adapters.outbound.persistence.clickhouse.gateway import (
    ClickHouseConnectGateway,
)

cfg = load_market_data_runtime_config(CONFIG_PATH)
print("Loaded market_data config:", CONFIG_PATH)
print("market_ids:", cfg.market_ids())

settings = ClickHouseSettingsLoader(os.environ).load()
client = _clickhouse_client(settings)
gateway = ClickHouseConnectGateway(client)

print("ClickHouse database:", settings.database)


ModuleNotFoundError: No module named 'apps'

In [ ]:
# HTTP клиент (requests) – адаптер common_http (как у тебя реализовано)
from trading.contexts.market_data.adapters.outbound.clients.common_http.requests_client import (
    RequestsHttpClient,
)

# REST source (как у тебя в проекте)
from trading.contexts.market_data.adapters.outbound.clients.rest_candle_ingest_source import (
    RestCandleIngestSource,
)
from trading.contexts.market_data.adapters.outbound.persistence.clickhouse.canonical_candle_index_reader import (  # noqa: E501
    ClickHouseCanonicalCandleIndexReader,
)
from trading.contexts.market_data.adapters.outbound.persistence.clickhouse.raw_kline_writer import (
    ClickHouseRawKlineWriter,
)
from trading.platform.time.system_clock import SystemClock

clock = SystemClock()

index = ClickHouseCanonicalCandleIndexReader(gateway=gateway, database=settings.database)
writer = ClickHouseRawKlineWriter(gateway=gateway, database=settings.database)

http = RequestsHttpClient()
source = RestCandleIngestSource(cfg=cfg, http=http, clock=clock)

print("Wiring done.")


ModuleNotFoundError: No module named 'trading.contexts.market_data.adapters.outbound.clients.common_http.requests_client'

In [ ]:
from trading.contexts.market_data.application.use_cases.rest_catchup_1m import RestCatchUp1mUseCase
from trading.shared_kernel.primitives import InstrumentId, MarketId, Symbol

max_days = int(MAX_DAYS_PER_INSERT or cfg.backfill.max_days_per_insert)

uc = RestCatchUp1mUseCase(
    index=index,
    source=source,
    writer=writer,
    clock=clock,
    max_days_per_insert=max_days,
    batch_size=int(BATCH_SIZE),
    ingest_id=INGEST_ID,
)

inst = InstrumentId(MarketId(int(MARKET_ID)), Symbol(str(SYMBOL)))
report = uc.run(inst)

print("REST catch-up report:")
print(json.dumps(asdict(report), ensure_ascii=False, indent=2))

if REPORT_JSON_PATH is not None:
    REPORT_JSON_PATH.parent.mkdir(parents=True, exist_ok=True)
    REPORT_JSON_PATH.write_text(json.dumps(asdict(report), ensure_ascii=False, indent=2), encoding="utf-8")  # noqa: E501
    print("Saved report to:", REPORT_JSON_PATH)


NameError: name 'cfg' is not defined

In [ ]:
# Reader для ref_instruments (должен уже быть в проекте)
from trading.contexts.market_data.adapters.outbound.persistence.clickhouse.ref_instruments_reader import (  # noqa: E501
    ClickHouseInstrumentRefReader,
)
from trading.shared_kernel.primitives import InstrumentId, MarketId, Symbol

ref_reader = ClickHouseInstrumentRefReader(gateway=gateway, database=settings.database)

enabled = ref_reader.enabled_instruments()
print("Enabled instruments:", len(enabled))

max_days = int(MAX_DAYS_PER_INSERT or cfg.backfill.max_days_per_insert)

uc = RestCatchUp1mUseCase(
    index=index,
    source=source,
    writer=writer,
    clock=clock,
    max_days_per_insert=max_days,
    batch_size=int(BATCH_SIZE),
    ingest_id=INGEST_ID,
)

reports = []
for inst in enabled:
    rep = uc.run(inst)
    reports.append(rep)
    print(f"{inst}: total_rows_written={rep.total_rows_written} lag_to_end_s={rep.lag_to_end_s:.3f}")  # noqa: E501

# агрегат по запуску
total_rows = sum(r.total_rows_written for r in reports)
print("TOTAL instruments:", len(reports))
print("TOTAL rows written:", total_rows)


ModuleNotFoundError: No module named 'trading.contexts.market_data.adapters.outbound.persistence.clickhouse.ref_instruments_reader'